In [12]:
# Useful starting lines
%timeit
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from preprocessing import *
from train_tune import *
from proj1_helpers import *

%load_ext autoreload
%autoreload 2
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 

#loading data
from preprocessing import cluster_preprocessing_train
y, tx_train, ids = load_csv_data(DATA_TRAIN_PATH)
names= get_feature_names(DATA_TRAIN_PATH)
name2num,num2name = mapping(DATA_TRAIN_PATH)
#Train preprocessing
print("THIS VERSION REMOVES THE 29TH COLUMN FROM CLUSTER0")
from preprocessing import cluster_preprocessing_train
tx0, y0, tx1, y1, tx2, y2, tx3, y3, id0, id1, id2, id3 = cluster_preprocessing_train(tx_train,y,num2name,f="median")

#K-fold CV
k_fold=10

lambdas = np.logspace(-9,-3,8)
degrees = np.array(range(1,12))

print("#======== CV for Cluster 0 ========#")
w_opt0, d_opt0, la0, train0, test0 = crossval_ridge_gridsearch(y0,tx0,k_fold,
                                                             lambdas,degrees,
                                                             loss=True)
cv_viz(d_opt0,lambdas,train0[d_opt0-1,:],test0[d_opt0-1,:],save="ridge_clust0finalmedian")
#----------------1
print("#======== CV for Cluster 1 ========#")
w_opt1, d_opt1, la1, train1, test1 = crossval_ridge_gridsearch(y1,tx1,k_fold,
                                                             lambdas,degrees,
                                                             loss=True)
cv_viz(d_opt1,lambdas,train1[d_opt1-1,:],test0[d_opt1-1,:],save="ridge_clust1finalmedian")

#----------------2
print("#======== CV for Cluster 2 ========#")
w_opt2, d_opt2, la2, train2, test2 = crossval_ridge_gridsearch(y2,tx2,k_fold,
                                                             lambdas,degrees,
                                                             loss=True)
cv_viz(d_opt2,lambdas,train2[d_opt2-1,:],test0[d_opt2-1,:],save="ridge_clust2finalmedian")

#----------------3
print("#======== CV for Cluster 3 ========#")
w_opt3, d_opt3, la3, train3, test3 = crossval_ridge_gridsearch(y3,tx3,k_fold,
                                                             lambdas,degrees,
                                                             loss=True)
cv_viz(d_opt3,lambdas,train3[d_opt3-1,:],test0[d_opt3-1,:],save="ridge_clust3finalmedian")

degs=[d_opt0,d_opt1,d_opt2,d_opt3]
####

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
THIS VERSION REMOVES THE 29TH COLUMN FROM CLUSTER0
PREPROCESSING TRAIN DATA 
 Clustering w.r.t. to PRI_jet_num numbers
Prediction targets detected. Using a training set. 
 Returning clusterized dataset and targets. 

Getting indices of columns to remove before taking log.
10 features deleted
7 features deleted
0 features deleted
0 features deleted
Taking the log of selected features

 Removing features where all rows are -999 or 0. Returning indices for later
Replacing -999 values with median
Standardizing : Setting mean to 0 and variance to 1
Preprocessing done
#======== CV for Cluster 0 ========#
Iterating. Testing 8 lambdas for current degree = 1
Iterating. Testing 8 lambdas for current degree = 2


KeyboardInterrupt: 

##### **Sanity check**

In [8]:
#Veryfing we get the correct results
tx_poly0 = build_poly(tx0,d_opt0)
w_test0, _ = ridge_regression(y0, tx_poly0, la0)

tx_poly1 = build_poly(tx1,d_opt1)
w_test1, _ = ridge_regression(y1, tx_poly1, la1)

tx_poly2 = build_poly(tx2, d_opt2)
w_test2, _ = ridge_regression(y2, tx_poly2, la2)

tx_poly3 = build_poly(tx3,d_opt3)
w_test3, _ = ridge_regression(y3, tx_poly3, la3)

print("hoping for the best")
print(w_opt0 == w_test0,"\n")
print(w_opt1 == w_test1,"\n")
print(w_opt2 == w_test2,"\n")
print(w_opt3 == w_test3,"\n")
print(degs)
print(w_opt0.shape, w_opt1.shape, w_opt2.shape, w_opt3.shape)
print(tx0.shape, tx1.shape, tx2.shape, tx3.shape)
print(build_poly(tx0,9).shape, build_poly(tx1,5).shape,build_poly(tx2,11).shape,build_poly(tx3,11).shape)
print(test0.shape, test1.shape, test2.shape, test3.shape)

hoping for the best
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True 

In [74]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
OUTPUT_PATH = '../finalsubmit/' # TODO: fill in desired name of output file for submission
#Test preprocessing
test0, i0, test1, i1, test2, i2, test3, i3 = cluster_preprocessing_test(tX_test, id0, id1, id2,id3, [9,5,11,11], degs,f="median")
#Prediction
yclusterpred_opt = cluster_predict(w_opt0,w_opt1,w_opt2,w_opt3,
                               test0,test1,test2,test3,
                               i0,i1,i2,i3,how="normal")

create_csv_submission(ids_test, yclusterpred_opt, OUTPUT_PATH+"CV_RIDGE_CORRECTEDLOGFEATURESwMedian.csv")


best = "degs = {},{},{},{} \n lambdas = {:e}, \n {:e}, \n {:e} \n {:e}".format(d_opt0,d_opt1,d_opt2,d_opt3,la0,la1,la2,la3)
file1 = open("bestparams_ridge_CORRECTEDLOGFEATURESwMedian","w") 
file1.write(best)
file1.close()

Normal prediction
